In [1]:
import pandas as pd
import numpy as np
import seaborn  as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math


### 01) 데이터 전처리

In [2]:
final_df = pd.read_csv('../data/SEOUL_CLUSTER.csv')
final_df

,상권_코드_명,상권_코드,기준_년_코드,기준_분기_코드,유사_업종_점포_수,개업_율,폐업_율,주중_매출_비율,주말_매출_비율,남성_매출_비율,...,연령대_3040_매출_비율,연령대_5060_매출_비율,집객시설_수,교통_인프라,총 상주인구 수,총_직장_인구_수,총_생활인구_수,월_평균_소득_금액,경영_위기_비율,클러스터
0,DMC(디지털미디어시티),2120098,2022,1,766,0.031353,0.016502,0.882753,0.117247,0.532487,...,0.583242,0.249914,72,24,1285,8846.0,318202,3567709.0,0.150943,3
1,DMC(디지털미디어시티),2120098,2022,2,768,0.042763,0.032895,0.874108,0.125892,0.529133,...,0.593012,0.231312,72,24,1285,8846.0,358782,3567709.0,0.207547,3
2,DMC(디지털미디어시티),2120098,2022,3,774,0.035714,0.027597,0.881428,0.118572,0.526975,...,0.587134,0.227869,72,24,1285,8846.0,352688,3567709.0,0.142857,3
3,DMC(디지털미디어시티),2120098,2022,4,778,0.041935,0.037097,0.868980,0.131020,0.527500,...,0.584706,0.232020,72,24,1285,8846.0,355095,3567709.0,0.291667,3
4,가락시장역,2120234,2022,1,1203,0.016393,0.022769,0.794609,0.205391,0.546158,...,0.389655,0.510317,62,3,2694,10268.0,1430185,3494644.0,0.235294,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,홍대입구역(홍대),2120103,2022,4,2979,0.033623,0.030369,0.617551,0.382449,0.431874,...,0.371155,0.127036,91,22,4789,9580.0,3897068,2934520.0,0.189474,0
448,화곡역,2120120,2022,1,636,0.024074,0.016667,0.761005,0.238995,0.485096,...,0.451777,0.350755,88,16,2748,927.0,1102013,2624496.0,0.166667,1
449,화곡역,2120120,2022,2,634,0.036969,0.038817,0.744602,0.255398,0.489530,...,0.463911,0.327942,88,16,2748,927.0,1168431,2624496.0,0.473684,1
450,화곡역,2120120,2022,3,627,0.016822,0.029907,0.765109,0.234891,0.494202,...,0.445831,0.353580,88,16,2748,927.0,1163547,2624496.0,0.380952,1


In [3]:
final_df.drop(['기준_년_코드', '기준_분기_코드', '상권_코드', '상권_코드_명'], axis=1, inplace=True)
final_df

,유사_업종_점포_수,개업_율,폐업_율,주중_매출_비율,주말_매출_비율,남성_매출_비율,여성_매출_비율,연령대_1020_매출_비율,연령대_3040_매출_비율,연령대_5060_매출_비율,집객시설_수,교통_인프라,총 상주인구 수,총_직장_인구_수,총_생활인구_수,월_평균_소득_금액,경영_위기_비율,클러스터
0,766,0.031353,0.016502,0.882753,0.117247,0.532487,0.467513,0.166844,0.583242,0.249914,72,24,1285,8846.0,318202,3567709.0,0.150943,3
1,768,0.042763,0.032895,0.874108,0.125892,0.529133,0.470867,0.175676,0.593012,0.231312,72,24,1285,8846.0,358782,3567709.0,0.207547,3
2,774,0.035714,0.027597,0.881428,0.118572,0.526975,0.473025,0.184997,0.587134,0.227869,72,24,1285,8846.0,352688,3567709.0,0.142857,3
3,778,0.041935,0.037097,0.868980,0.131020,0.527500,0.472500,0.183274,0.584706,0.232020,72,24,1285,8846.0,355095,3567709.0,0.291667,3
4,1203,0.016393,0.022769,0.794609,0.205391,0.546158,0.453842,0.100028,0.389655,0.510317,62,3,2694,10268.0,1430185,3494644.0,0.235294,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,2979,0.033623,0.030369,0.617551,0.382449,0.431874,0.568126,0.501809,0.371155,0.127036,91,22,4789,9580.0,3897068,2934520.0,0.189474,0
448,636,0.024074,0.016667,0.761005,0.238995,0.485096,0.514904,0.197469,0.451777,0.350755,88,16,2748,927.0,1102013,2624496.0,0.166667,1
449,634,0.036969,0.038817,0.744602,0.255398,0.489530,0.510470,0.208147,0.463911,0.327942,88,16,2748,927.0,1168431,2624496.0,0.473684,1
450,627,0.016822,0.029907,0.765109,0.234891,0.494202,0.505798,0.200589,0.445831,0.353580,88,16,2748,927.0,1163547,2624496.0,0.380952,1


In [15]:
cluster0_df = final_df[final_df['클러스터'] == 0].copy()
cluster1_df = final_df[final_df['클러스터'] == 1].copy()
cluster2_df = final_df[final_df['클러스터'] == 2].copy()
cluster3_df = final_df[final_df['클러스터'] == 3].copy()

In [16]:
cluster0_df.drop('클러스터', axis=1, inplace=True)
cluster1_df.drop('클러스터', axis=1, inplace=True)
cluster2_df.drop('클러스터', axis=1, inplace=True)
cluster3_df.drop('클러스터', axis=1, inplace=True)

In [17]:
cluster3_df

,유사_업종_점포_수,개업_율,폐업_율,주중_매출_비율,주말_매출_비율,남성_매출_비율,여성_매출_비율,연령대_1020_매출_비율,연령대_3040_매출_비율,연령대_5060_매출_비율,집객시설_수,교통_인프라,총 상주인구 수,총_직장_인구_수,총_생활인구_수,월_평균_소득_금액,경영_위기_비율
0,766,0.031353,0.016502,0.882753,0.117247,0.532487,0.467513,0.166844,0.583242,0.249914,72,24,1285,8846.0,318202,3567709.0,0.150943
1,768,0.042763,0.032895,0.874108,0.125892,0.529133,0.470867,0.175676,0.593012,0.231312,72,24,1285,8846.0,358782,3567709.0,0.207547
2,774,0.035714,0.027597,0.881428,0.118572,0.526975,0.473025,0.184997,0.587134,0.227869,72,24,1285,8846.0,352688,3567709.0,0.142857
3,778,0.041935,0.037097,0.868980,0.131020,0.527500,0.472500,0.183274,0.584706,0.232020,72,24,1285,8846.0,355095,3567709.0,0.291667
16,1243,0.018389,0.016637,0.811165,0.188835,0.476363,0.523637,0.159252,0.503226,0.337522,88,10,1522,7726.0,1562580,3950137.0,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,1359,0.023846,0.020000,0.680814,0.319186,0.416708,0.583292,0.212277,0.547885,0.239839,78,4,976,9363.0,1260738,3402858.0,0.255556
428,1059,0.027254,0.023061,0.719238,0.280762,0.509212,0.490788,0.248069,0.508429,0.243501,77,17,6927,1932.0,1378358,4649382.0,0.125000
429,1056,0.023085,0.019937,0.725262,0.274738,0.515811,0.484189,0.255020,0.498558,0.246423,77,17,6927,1932.0,1513316,4649382.0,0.071429
430,1051,0.021075,0.024236,0.737214,0.262786,0.546283,0.453717,0.249854,0.490906,0.259241,77,17,6927,1932.0,1531543,4649382.0,0.102564


### 02) 모델 학습

##### cluster 0

In [20]:
x1 = cluster0_df.drop('경영_위기_비율', axis=1)
y1 = cluster0_df['경영_위기_비율']
x1_train, x1_valid, y1_train, y1_valid= train_test_split(x1, y1, test_size=0.2, random_state=0)

In [1]:
params = {
    'n_estimators':(200, 300, 400),
    'max_depth' : (4 ,6, 8, 10, 12, 14, 16),
    'min_samples_leaf' : (4, 6, 8, 10, 12, 14, 16),
    'min_samples_split' : (4, 6, 8, 10, 12, 14, 16)
}

rf = RandomForestRegressor(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf, param_grid=params, cv=10, n_jobs=-1)
grid_cv.fit(x1_train, y1_train)
 
 
print('최적 하이퍼 파라미터:', grid_cv.best_params_)
print('최적 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

NameError: name 'RandomForestRegressor' is not defined

In [ ]:
rf = RandomForestRegressor(random_state=0, max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=200)
rf.fit(x_train, y_train)

In [ ]:
# train rmse
train_predict = rf.predict(x_train)
print("RMSE':{}".format(math.sqrt(mean_squared_error(train_predict, y_train))))

# valid rmse
valid_predict = rf.predict(x_valid)
print("RMSE':{}".format(math.sqrt(mean_squared_error(train_predict, y_valid))))

In [ ]:
ftr_importances_values = rf.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index=x_train.columns)
ftr_top = ftr_importances.sort_values(ascending=False)[:20]
 
plt.figure(figsize=(8, 6))
sns.barplot(x=ftr_top, y=ftr_top.index)
plt.show()